In [ ]:
import csv
from pathlib import Path

from evaluate import evaluate, plot_results
from generate_data import generate_data
from training import train

## Data Generation

In [ ]:
total_error_budget = 0.1
number_of_randomly_generated_distributions = 1000
use_paper_data = True
use_zip_file_circuits = False

if use_paper_data:
    csv_path = Path("paper_data/logical_counts.csv")
    with Path.open(csv_path, encoding="utf-8", newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        logical_counts = [{k: int(v) for k, v in row.items()} for row in reader]
    data = generate_data(
        total_error_budget,
        number_of_randomly_generated_distributions,
        logical_counts=logical_counts,
    )

elif use_zip_file_circuits:
    zip_file_path = Path("paper_data") / "mqtbench.zip"
    if not zip_file_path.exists():
        msg = f"Paper data not found at {zip_file_path}"
        raise FileNotFoundError(msg)
    data = generate_data(
        total_error_budget,
        number_of_randomly_generated_distributions,
        path=zip_file_path,
    )
else:
    benchmark_defs = [
        ("ae", [3, 4, 5, 6, 7, 8, 9, 10]),
        ("dj", [3, 4, 5, 6, 7, 8, 9, 10]),
    ]
    data = generate_data(
        total_error_budget,
        number_of_randomly_generated_distributions,
        benchmarks_and_sizes=benchmark_defs,
    )

## Training

In [ ]:
model, X_test, Y_test = train(data)

## Evaluation

In [ ]:
Y_pred = model.predict(X_test)
product_diffs = evaluate(X_test, Y_pred, total_error_budget)
product_diffs_dataset = evaluate(X_test, Y_test, total_error_budget)

plot_results(product_diffs, product_diffs_dataset, legend=True, bin_width=4)